An example of searching on nested fields and values

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from elasticsearch.client import CatClient

In [2]:
elastic_db = Elasticsearch(['localhost:9200'])
indices_client = IndicesClient(elastic_db)
cat_client = CatClient(elastic_db)

In [3]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

health status index   uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   .kibana _RRwYu7rSd2H7CuBLtZ4dw   1   1          2            1     12.2kb         12.2kb



In [4]:
# Create the dest_idx with required settings and mappings
idx_structure = {
    "mappings": {
        "doc": {
            "properties": {
                "sku":{"type":"text"},
                "productFamily":{"type":"text"},
                "price":{"type":"integer"},
                "attributes":{
                    "type": "nested",
                    "properties":{
                        "vcpu":{"type":"integer"},
                        "memory":{"type":"integer"}
                    }
                }
            }
        }
    }
}

response = indices_client.create(index='nstd_idx',body=idx_structure)
print(response)

{'shards_acknowledged': True, 'index': 'nstd_idx', 'acknowledged': True}


In [5]:
# Add document 1
doc_body = {
    "sku": "sku1",
    "productFamily":"sku1_product",
    "price":25,
    "attributes":{
        "vcpu": "32",
        "memory": "264"
    }
}

response = elastic_db.index(index='nstd_idx', doc_type='doc', body=doc_body,refresh=True)
print(response)

{'result': 'created', '_primary_term': 1, 'forced_refresh': True, '_shards': {'failed': 0, 'total': 2, 'successful': 1}, '_index': 'nstd_idx', '_type': 'doc', '_version': 1, '_seq_no': 0, '_id': 'd31CQmUBOrnFUpGS4RUm'}


In [6]:
# Add document 2
doc_body = {
    "sku": "sku2",
    "productFamily":"sku2_product",
    "price":50,
    "attributes":
    {
        "vcpu": "64",
        "memory": "512"
    }
}

response = elastic_db.index(index='nstd_idx', doc_type='doc', body=doc_body,refresh=True)
print(response)

{'result': 'created', '_primary_term': 1, 'forced_refresh': True, '_shards': {'failed': 0, 'total': 2, 'successful': 1}, '_index': 'nstd_idx', '_type': 'doc', '_version': 1, '_seq_no': 0, '_id': 'eH1CQmUBOrnFUpGS4RX4'}


In [7]:
# Searching on sku
query_body = {
    "query": {
        "match":{
            "sku":"sku2"
        }
    }
}

response = elastic_db.search(index='nstd_idx', body=query_body, explain=False)
print(response['hits']['hits'][0])

{'_score': 0.2876821, '_source': {'price': 50, 'attributes': {'vcpu': '64', 'memory': '512'}, 'sku': 'sku2', 'productFamily': 'sku2_product'}, '_id': 'eH1CQmUBOrnFUpGS4RX4', '_index': 'nstd_idx', '_type': 'doc'}


In [8]:
# Searching on nested vcpu
query_body = {
    "query": {
        "nested":{
            "path":"attributes",
            "query":{
                "match":{
                    "attributes.vcpu":"64"
                }
            }
        }
    }
}

response = elastic_db.search(index='nstd_idx', body=query_body, explain=False)
print(response['hits']['hits'][0])

{'_score': 1.0, '_source': {'price': 50, 'attributes': {'vcpu': '64', 'memory': '512'}, 'sku': 'sku2', 'productFamily': 'sku2_product'}, '_id': 'eH1CQmUBOrnFUpGS4RX4', '_index': 'nstd_idx', '_type': 'doc'}


In [9]:
# Searching for a closest value in a nested field
query_body = {
    "query":{
        "nested":{
            "path":"attributes",
            "query":{
                "function_score":{
                    "query":{
                        "match_all":{}
                    },
                    "script_score":{
                        "script":{
                            "params":{"val":64},
                            "source":"-(Math.abs(params.val - doc['attributes.vcpu'].value))"
                        }
                    },
                    "boost_mode": "replace"
                }
            }
        }
    }
}

response = elastic_db.search(index='nstd_idx', body=query_body, explain=False)
print(response['hits']['hits'][0])

{'_score': 0.0, '_source': {'price': 50, 'attributes': {'vcpu': '64', 'memory': '512'}, 'sku': 'sku2', 'productFamily': 'sku2_product'}, '_id': 'eH1CQmUBOrnFUpGS4RX4', '_index': 'nstd_idx', '_type': 'doc'}


In [10]:
# Searching for a closest value in a nested field as well as a particular match
query_body = {
    "query":{
        "bool":{
            "must":[
                {
                    "match":{"sku":"sku2"}
                },
                {
                    "nested":{
                        "path":"attributes",
                        "query":{
                            "function_score":{
                                "query":{
                                    "match_all":{}
                                },
                                "script_score":{
                                    "script":{
                                        "params":{"val":64},
                                        "source":"-(Math.abs(params.val - doc['attributes.vcpu'].value))"
                                    }
                                },
                                "boost_mode": "replace"
                            }
                        }
                    }
                }
            ]
        }
    }
}

response = elastic_db.search(index='nstd_idx', body=query_body, explain=False)
print(response['hits']['hits'][0])

{'_score': 0.2876821, '_source': {'price': 50, 'attributes': {'vcpu': '64', 'memory': '512'}, 'sku': 'sku2', 'productFamily': 'sku2_product'}, '_id': 'eH1CQmUBOrnFUpGS4RX4', '_index': 'nstd_idx', '_type': 'doc'}


In [11]:
# Searching and combining multiple scores
query_body = {
    "query":{
        "bool":{
            "must":[
                {
                    "match":{"sku":"sku2"}
                },
                {
                    "nested":{
                        "path":"attributes",
                        "query":{
                            "function_score":{
                                "query":{
                                    "match_all":{}
                                },
                                "functions":[
                                    {
                                        "gauss":{
                                            "attributes.vcpu":{
                                                "origin":"64",
                                                "scale":"1"
                                            }
                                        }
                                    },
                                    {
                                        "gauss":{
                                            "attributes.memory":{
                                                "origin":"512",
                                                "scale":"1"
                                            }
                                        }
                                    }
                                ],
                                "score_mode":"sum",
                                "boost_mode": "replace"
                            }
                        }
                    }
                }
            ]
        }
    }
}

response = elastic_db.search(index='nstd_idx', body=query_body, explain=False)
print(response['hits']['hits'][0])

{'_score': 2.287682, '_source': {'price': 50, 'attributes': {'vcpu': '64', 'memory': '512'}, 'sku': 'sku2', 'productFamily': 'sku2_product'}, '_id': 'eH1CQmUBOrnFUpGS4RX4', '_index': 'nstd_idx', '_type': 'doc'}
